# 개선점 정리

1. Decoder-Only 아키텍처로 변경해야 한다.
2. Masked Self-Attention을 해야 한다.
3. supervised learning이 아닌 pre-training 방식으로 변경한다.

---
## 주요 변경점 정리:
"""
Transformer → GPT 전처리 변경사항:

1. **Encoder-Decoder → Decoder-Only**
   - 기존: enc_input, dec_input, target 분리
   - GPT: input_ids, target_ids (시퀀스를 한 칸씩 밀어서 생성)

2. **Cross-Attention 제거**
   - 기존: enc_padding_mask, dec_padding_mask 별도 관리
   - GPT: attention_mask 하나로 통합

3. **Causal Masking 준비**
   - 데이터셋에서는 어텐션 마스크만 준비
   - 실제 causal masking은 모델에서 구현

4. **연속 텍스트 처리**
   - 기존: Q-A 페어를 별도 처리
   - GPT: Q-A를 연결한 연속 텍스트로 처리

5. **Language Modeling 방식**
   - 다음 토큰 예측을 위한 input/target 시프트
"""

# 데이터 전처리

## EDA

In [3]:
!pip install sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 11.8 MB/s eta 0:00:00


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import sentencepiece as spm

import math
import os
import re
import urllib.request
import zipfile
import numpy as np
import matplotlib.pyplot as plt

In [5]:
cd ~/work/transformer_chatbot/data/

/home/jovyan/work/transformer_chatbot/data


In [6]:
import pandas as pd

df = pd.read_csv("ChatbotData.csv")

In [7]:
df.head()       # 앞부분 5행 보기

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [11]:
print(df.shape)        # (행 개수, 열 개수)
print('======================================')
print(df.columns)      # 컬럼명 확인
print('======================================')
df.info()       # 컬럼 정보, 데이터 타입, 결측치 확인

(11823, 3)
Index(['Q', 'A', 'label'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11823 entries, 0 to 11822
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       11823 non-null  object
 1   A       11823 non-null  object
 2   label   11823 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 277.2+ KB


## 전처리 클래스

In [12]:
# 전처리 함수
def preprocess_sentence(sentence):
  # 입력받은 sentence를 소문자로 변경하고 문장 앞 뒤의 공백을 제거
  sentence = sentence.lower().strip()

  # 단어와 구두점(punctuation) 사이의 거리를 만듭니다.
  # 예를 들어서 "12시가 땡!" => "12시가 땡 !"와 같이
  # 땡과 느낌표 사이에 거리를 만듭니다.
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)

  # 연속된 공백을 하나의 공백으로 축소
  sentence = re.sub(r'[" "]+', " ", sentence)

  #한글, 영어, 숫자, 구두점만 남기고 나머지 제고
  sentence = re.sub(r"[^가-힣a-zA-Z0-9?.!,]+", " ", sentence)
  sentence = sentence.strip() # 앞 뒤 공백 다시 제거
  return sentence
print("슝=3")

슝=3


In [13]:
# GPT용 말뭉치 생성 (언어모델링을 위한 연속 텍스트)
def create_gpt_corpus(df, corpus_file="gpt_corpus.txt"):
    """
    GPT는 연속적인 텍스트로 학습하므로 Q-A를 연결하여 하나의 텍스트로 만듦
    """
    with open(corpus_file, 'w', encoding='utf-8') as f:
        for _, row in df.iterrows():
            # Q와 A를 연결하되, 구분자를 추가
            q = preprocess_sentence(str(row['Q']))
            a = preprocess_sentence(str(row['A']))
            # GPT용: 질문과 답변을 하나의 시퀀스로 연결
            combined_text = q + " " + a
            f.write(combined_text + "\n")
    
    return corpus_file

In [14]:
# SentencePiece 토크나이저 학습 (GPT용)
import sentencepiece as spm

def train_gpt_tokenizer(corpus_file, vocab_size=8000):
    """
    GPT용 토크나이저 학습
    """
    spm.SentencePieceTrainer.Train(
        input=corpus_file,
        model_prefix="gpt_tokenizer",
        vocab_size=vocab_size,
        character_coverage=1.0,
        model_type="bpe",
        max_sentence_length=999999,
        # GPT용 특수 토큰 설정
        bos_id=1,   # <s> 시작 토큰
        eos_id=2,   # </s> 종료 토큰  
        pad_id=0,   # <pad> 패딩 토큰
        unk_id=3    # <unk> 미지 토큰
    )

## 데이터셋 만들기 클래스

In [15]:
# GPT용 Dataset 클래스
from torch.utils.data import Dataset
import torch

class GPTDataset(Dataset):
    def __init__(self, texts, sp, max_length=512):
        """
        GPT용 Dataset - Decoder-Only 아키텍처에 맞게 수정
        
        Args:
            texts: 학습할 텍스트 리스트
            sp: sentencepiece processor
            max_length: 최대 시퀀스 길이
        """
        super().__init__()
        self.sp = sp
        self.max_length = max_length
        self.data = []
        
        # 특수 토큰 ID
        self.pad_id = sp.pad_id() if sp.pad_id() >= 0 else 0
        self.bos_id = sp.bos_id() if sp.bos_id() >= 0 else 1
        self.eos_id = sp.eos_id() if sp.eos_id() >= 0 else 2
        
        # 텍스트 전처리
        for text in texts:
            # 1) 토크나이즈
            token_ids = sp.EncodeAsIds(text)
            
            # 2) BOS, EOS 토큰 추가
            token_ids = [self.bos_id] + token_ids + [self.eos_id]
            
            # 3) 길이 제한
            if len(token_ids) > self.max_length:
                token_ids = token_ids[:self.max_length]
            
            # 4) 충분히 긴 시퀀스만 사용 (너무 짧으면 학습 효과 적음)
            if len(token_ids) < 10:  # 최소 길이 체크
                continue
                
            # 5) GPT용 입력/타겟 생성
            # input: [BOS, token1, token2, ..., tokenN]
            # target: [token1, token2, ..., tokenN, EOS]
            input_ids = token_ids[:-1]   # 마지막 토큰 제외
            target_ids = token_ids[1:]   # 첫 번째 토큰 제외
            
            # 6) 패딩 (필요한 경우)
            input_length = len(input_ids)
            if input_length < self.max_length - 1:  # -1은 target과 길이 맞추기 위함
                pad_length = (self.max_length - 1) - input_length
                input_ids = input_ids + [self.pad_id] * pad_length
                target_ids = target_ids + [self.pad_id] * pad_length
            
            # 7) 어텐션 마스크 생성 (패딩 마스크)
            attention_mask = [1 if token_id != self.pad_id else 0 for token_id in input_ids]
            
            self.data.append({
                "input_ids": input_ids,
                "target_ids": target_ids,
                "attention_mask": attention_mask,
                "length": input_length  # 실제 시퀀스 길이 (패딩 제외)
            })
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        return (
            torch.tensor(sample["input_ids"], dtype=torch.long),
            torch.tensor(sample["target_ids"], dtype=torch.long),
            torch.tensor(sample["attention_mask"], dtype=torch.long),
            torch.tensor(sample["length"], dtype=torch.long)
        )


In [16]:
def prepare_gpt_data(csv_path):
    """
    GPT 학습을 위한 전체 데이터 준비 과정
    """
    from pathlib import Path
    import pandas as pd
    
    # 1) CSV 로드 및 전처리
    df = pd.read_csv(csv_path)
    df['Q'] = df['Q'].astype(str).apply(preprocess_sentence)
    df['A'] = df['A'].astype(str).apply(preprocess_sentence)
    
    # 2) GPT용 말뭉치 생성
    corpus_file = create_gpt_corpus(df)
    
    # 3) 토크나이저 학습
    train_gpt_tokenizer(corpus_file)
    
    # 4) 토크나이저 로드
    sp = spm.SentencePieceProcessor()
    sp.Load("gpt_tokenizer.model")
    
    # 5) 학습용 텍스트 준비
    texts = []
    with open(corpus_file, 'r', encoding='utf-8') as f:
        texts = [line.strip() for line in f if line.strip()]
    
    # 6) Dataset 생성
    dataset = GPTDataset(texts, sp, max_length=256)
    
    return dataset, sp

In [50]:
import sentencepiece as spm
sp = spm.SentencePieceProcessor(model_file="gpt_tokenizer.model")

In [51]:
vocab_size = sp.vocab_size()

# 모델 만들기

In [63]:
# 퀘스트3
class GPT(nn.Module):
    def __init__(self,
                 vocab_size,
                 num_layers,      # 디코더 블록 수
                 units,           # FFN 차원 (4*d_model 권장)
                 d_model,         # 임베딩 차원
                 num_heads,       # 헤드 수
                 max_seq_len=512,
                 dropout=0.1):
        super().__init__()
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.max_seq_len = max_seq_len

        # 임베딩
        self.token_embedding = nn.Embedding(vocab_size, d_model)
        self.position_embedding = nn.Embedding(max_seq_len, d_model)
        self.dropout = nn.Dropout(dropout)

        # 디코더 블록들
        self.decoder_blocks = nn.ModuleList([
            GPTDecoderBlock(d_model=d_model, num_heads=num_heads, ff_dim=units, dropout=dropout)
            for _ in range(num_layers)
        ])

        # 최종 LN + 출력
        self.layer_norm = nn.LayerNorm(d_model)
        self.final_linear = nn.Linear(d_model, vocab_size)

        # 가중치 초기화
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def create_causal_mask(self, seq_len, device, batch_size=None):
        # (1,1,S,S) causal mask
        causal = torch.tril(torch.ones(seq_len, seq_len, device=device))
        causal = causal.unsqueeze(0).unsqueeze(0)  # (1,1,S,S)
        if batch_size is not None:
            causal = causal.expand(batch_size, -1, -1, -1)  # (B,1,S,S)로 브로드캐스트 대비
        return causal

    def forward(self, input_ids, attention_mask=None):
        """
        input_ids: (B, S)
        attention_mask: (B, S)  1=유효토큰, 0=패딩
        """
        B, S = input_ids.shape
        device = input_ids.device

        pos = torch.arange(S, device=device).unsqueeze(0).expand(B, S)

        x = self.token_embedding(input_ids) + self.position_embedding(pos)
        x = self.dropout(x)

        # 마스크 만들기
        causal = self.create_causal_mask(S, device)  # (1,1,S,S)
        if attention_mask is not None:
            # (B,1,1,S)로 확장
            pad = attention_mask.unsqueeze(1).unsqueeze(1).to(dtype=causal.dtype)
            attn_mask = causal * pad  # (B,1,S,S)로 브로드캐스트
        else:
            attn_mask = causal  # (1,1,S,S)

        for block in self.decoder_blocks:
            x = block(x, attn_mask=attn_mask)

        x = self.layer_norm(x)
        logits = self.final_linear(x)  # (B,S,V)
        return logits

In [62]:
# GPT용 디코더 블록 (Cross-Attention 제거)
class GPTDecoderBlock(nn.Module):
    def __init__(self, d_model, num_heads, ff_dim, dropout=0.1):
        super().__init__()
        # Pre-LN
        self.ln1 = nn.LayerNorm(d_model)
        self.mha = MultiHeadAttention(d_model=d_model, num_heads=num_heads, dropout=dropout)
        self.ln2 = nn.LayerNorm(d_model)
        self.ff = nn.Sequential(
            nn.Linear(d_model, ff_dim),
            nn.GELU(),
            nn.Linear(ff_dim, d_model),
            nn.Dropout(dropout),
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, attn_mask=None):
        # Self-Attention
        h = self.ln1(x)
        h = self.mha(h, attn_mask=attn_mask)
        x = x + self.dropout(h)
        # FFN
        h = self.ln2(x)
        h = self.ff(h)
        x = x + h
        return x

In [61]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1):
        super().__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

        self.attn_dropout = nn.Dropout(dropout)
        self.proj_dropout = nn.Dropout(dropout)

    def _split_heads(self, x):
        # x: (B, S, D) -> (B, H, S, Dh)
        B, S, D = x.shape
        x = x.view(B, S, self.num_heads, self.head_dim)
        return x.permute(0, 2, 1, 3)

    def _combine_heads(self, x):
        # x: (B, H, S, Dh) -> (B, S, D)
        B, H, S, Dh = x.shape
        x = x.permute(0, 2, 1, 3).contiguous()
        return x.view(B, S, H * Dh)

    def forward(self, x, attn_mask=None):
        """
        x: (B, S, D)
        attn_mask: (B, 1, S, S) 또는 (1, 1, S, S); 1=keep, 0=mask
        """
        Q = self._split_heads(self.W_q(x))
        K = self._split_heads(self.W_k(x))
        V = self._split_heads(self.W_v(x))

        # (B, H, S, Dh) x (B, H, Dh, S) -> (B, H, S, S)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.head_dim)

        if attn_mask is not None:
            # mask가 float이면 그대로 쓰고, bool/int면 0인 곳만 -inf
            if attn_mask.dtype != torch.float32 and attn_mask.dtype != torch.float16 and attn_mask.dtype != torch.bfloat16:
                mask = (attn_mask == 0)
                scores = scores.masked_fill(mask, float('-inf'))
            else:
                scores = scores.masked_fill(attn_mask == 0, float('-inf'))

        attn = F.softmax(scores, dim=-1)
        attn = self.attn_dropout(attn)

        out = torch.matmul(attn, V)            # (B, H, S, Dh)
        out = self._combine_heads(out)         # (B, S, D)
        out = self.W_o(out)
        out = self.proj_dropout(out)
        return out

In [39]:
# 퀘스트 4: 모델 생성 및 학습 준비
def create_gpt_model(vocab_size, config=None):
    """
    GPT 모델 생성
    """
    if config is None:
        # GPT-1 기본 설정
        config = {
            'vocab_size': vocab_size,
            'num_layers': 12,      # GPT-1: 12층
            'units': 3072,         # GPT-1: 4 * d_model 
            'd_model': 768,        # GPT-1: 768
            'num_heads': 12,       # GPT-1: 12헤드
            'max_seq_len': 512,    # 최대 시퀀스 길이
            'dropout': 0.1
        }
    
    model = GPT(**config)
    return model


In [46]:
model = create_gpt_model(8000, config=None)

In [41]:
# 손실 함수 (Language Modeling용)
def gpt_loss_function(logits, targets, attention_mask=None):
    """
    GPT 언어모델링 손실 계산
    
    Args:
        logits: (batch_size, seq_len, vocab_size)
        targets: (batch_size, seq_len)
        attention_mask: (batch_size, seq_len) - 패딩 위치 제외용
    """
    # Cross-entropy loss
    loss_fn = nn.CrossEntropyLoss(reduction='none')  # 각 위치별 손실 계산
    
    # logits을 2D로, targets을 1D로 reshape
    logits_flat = logits.view(-1, logits.size(-1))  # (batch_size * seq_len, vocab_size)
    targets_flat = targets.view(-1)                 # (batch_size * seq_len,)
    
    # 손실 계산
    losses = loss_fn(logits_flat, targets_flat)     # (batch_size * seq_len,)
    losses = losses.view(targets.size())            # (batch_size, seq_len)
    
    # 패딩 위치 제외
    if attention_mask is not None:
        losses = losses * attention_mask.float()
        # 실제 토큰 개수로 평균
        return losses.sum() / attention_mask.sum()
    else:
        return losses.mean()

## 학습하기

In [42]:
def main():
    # 1. 모델 생성
    vocab_size = 8000  # SentencePiece 토크나이저 크기
    model = create_gpt_model(vocab_size)
    
    # 2. 모델 요약 출력
    print("=== GPT Model Summary ===")
    print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")
    print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
    
    # 3. 예시 입력으로 테스트
    batch_size = 2
    seq_len = 10
    
    # 더미 입력 생성
    input_ids = torch.randint(0, vocab_size, (batch_size, seq_len))
    attention_mask = torch.ones(batch_size, seq_len)
    
    # Forward pass
    with torch.no_grad():
        logits = model(input_ids, attention_mask)
        print(f"Input shape: {input_ids.shape}")
        print(f"Output shape: {logits.shape}")
    
    # 4. 손실 계산 예시
    targets = torch.randint(0, vocab_size, (batch_size, seq_len))
    loss = gpt_loss_function(logits, targets, attention_mask)
    print(f"Loss: {loss.item():.4f}")
    
    return model


In [43]:
def train(model, dataloader, optimizer, loss_function, scheduler, num_epochs, device):
    """
    GPT 모델 학습 함수
    
    Args:
        model: GPT 모델
        dataloader: 데이터로더
        optimizer: 옵티마이저
        loss_function: 손실 함수
        scheduler: 학습률 스케줄러
        num_epochs: 에폭 수
        device: 디바이스 (cuda/cpu)
    """
    model = model.to(device)
    model.train()
    
    print(f"Training started - {num_epochs} epochs, {len(dataloader)} batches per epoch")
    
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        num_batches = 0
        
        for batch_idx, batch in enumerate(dataloader):
            # 배치를 device로 이동
            input_ids, target_ids, attention_mask, lengths = batch
            input_ids = input_ids.to(device)
            target_ids = target_ids.to(device)
            attention_mask = attention_mask.to(device)
            
            # Forward pass
            optimizer.zero_grad()
            logits = model(input_ids, attention_mask)
            loss = loss_function(logits, target_ids, attention_mask)
            
            # Backward pass
            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            # Update parameters
            optimizer.step()
            if scheduler is not None:
                scheduler.step()
            
            # Statistics
            epoch_loss += loss.item()
            num_batches += 1
            
            # Progress logging
            if batch_idx % 100 == 0:
                current_lr = scheduler.get_last_lr()[0] if scheduler else optimizer.param_groups[0]['lr']
                print(f"Epoch {epoch+1:2d}/{num_epochs}, "
                      f"Batch {batch_idx:3d}/{len(dataloader)}, "
                      f"Loss: {loss.item():.4f}, "
                      f"LR: {current_lr:.6f}")
        
        # Epoch summary
        avg_loss = epoch_loss / num_batches
        print(f"Epoch {epoch+1:2d} completed - Average Loss: {avg_loss:.4f}")
    
    print("Training completed!")

In [52]:
# 전체 설정 및 실행을 위한 헬퍼 함수
def setup_training():
    """
    학습에 필요한 모든 컴포넌트 준비
    """
    from pathlib import Path
    from torch.utils.data import DataLoader
    
    # 1. 데이터 준비
    print("Setting up data...")
    csv_path = Path.home() / "work/transformer_chatbot/data/ChatbotData.csv"
    dataset, sp = prepare_gpt_data(csv_path)
    
    # 2. 데이터로더
    dataloader = DataLoader(
        dataset, 
        batch_size=16,  # 배치 크기 조정
        shuffle=True,
        num_workers=2,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    # 3. 모델
    print("Setting up model...")
    vocab_size = sp.vocab_size()
    model = create_gpt_model(vocab_size)
    
    # 4. 옵티마이저
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=5e-4,
        betas=(0.9, 0.95),
        eps=1e-8,
        weight_decay=0.1
    )
    
    # 5. 스케줄러
    total_steps = len(dataloader) * 25  # 25 에폭 기준
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer,
        T_max=total_steps,
        eta_min=1e-6
    )
    
    # 6. 손실 함수
    loss_function = gpt_loss_function
    
    # 7. 디바이스
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    return model, dataloader, optimizer, loss_function, scheduler, device


In [64]:
model, dataloader, optimizer, loss_function, scheduler, device = setup_training()

Setting up data...


sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: gpt_corpus.txt
  input_format: 
  model_prefix: gpt_tokenizer
  model_type: BPE
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 999999
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 3
  bos_id: 1
  eos_id: 2
  pad_id: 0
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  differ

Setting up model...
Using device: cuda


In [66]:
%%time

train(
    model=model,
    dataloader=dataloader,
    optimizer=optimizer,
    loss_function=loss_function,
    scheduler=scheduler,
    num_epochs=1,  # 원하는 에폭 수
    device=device
)

Training started - 1 epochs, 628 batches per epoch
Epoch  1/1, Batch   0/628, Loss: 6.2800, LR: 0.000495
Epoch  1/1, Batch 100/628, Loss: 6.3250, LR: 0.000494
Epoch  1/1, Batch 200/628, Loss: 6.3284, LR: 0.000493
Epoch  1/1, Batch 300/628, Loss: 6.2017, LR: 0.000491
Epoch  1/1, Batch 400/628, Loss: 6.3782, LR: 0.000490
Epoch  1/1, Batch 500/628, Loss: 6.4750, LR: 0.000489
Epoch  1/1, Batch 600/628, Loss: 6.5008, LR: 0.000487
Epoch  1 completed - Average Loss: 6.3451
Training completed!
CPU times: user 7min 31s, sys: 1.18 s, total: 7min 32s
Wall time: 7min 31s


In [67]:
import torch

@torch.no_grad()
def decoder_inference(model, sentence, tokenizer, device='cpu',
                      max_len=50, bos_token=None, eos_token=None):
    """
    간단한 greedy search 기반 디코딩
    """
    model.eval()

    # 1) 입력 문장 → 토큰 ID
    input_ids = tokenizer.encode(sentence, out_type=int)
    if bos_token is not None:
        input_ids = [bos_token] + input_ids
    input_ids = torch.tensor([input_ids], dtype=torch.long, device=device)

    for _ in range(max_len):
        # 2) 모델 forward
        logits = model(input_ids)          # (1, seq_len, vocab_size)
        next_token_logits = logits[0, -1]  # 마지막 토큰에 대한 분포

        # 3) 가장 확률 높은 토큰 선택 (greedy)
        next_token = torch.argmax(next_token_logits, dim=-1).item()

        # 4) EOS 만나면 종료
        if eos_token is not None and next_token == eos_token:
            break

        # 5) 다음 토큰을 이어붙임
        input_ids = torch.cat(
            [input_ids, torch.tensor([[next_token]], device=device)], dim=1
        )

    return input_ids.squeeze(0).tolist()


In [68]:
def sentence_generation(model, sentence, tokenizer, device='cpu',
                        max_len=50, bos_token=None, eos_token=None):
    output_seq = decoder_inference(
        model, sentence, tokenizer, device=device,
        max_len=max_len, bos_token=bos_token, eos_token=eos_token
    )

    # 토큰을 문자열로 디코딩
    predicted_sentence = tokenizer.decode(
        [tok for tok in output_seq if tok < tokenizer.vocab_size()]
    )

    print("입력 :", sentence)
    print("출력 :", predicted_sentence)
    return predicted_sentence


In [73]:
test_sentence = "날씨 좋다"
sentence_generation(model, test_sentence, sp, device=device,
                    bos_token=sp.bos_id(), eos_token=sp.eos_id())

입력 : 날씨 좋다
출력 : 날씨 좋다이 안 안 안 해보세요 .


'날씨 좋다이 안 안 안 해보세요 .'